# 🏗️ CE49X: AI Trends in Civil Engineering
### 🚀 End-to-End Automated Data Analysis Pipeline

**Project Overview:**
This notebook integrates the entire research workflow, identifying the most prominent Artificial Intelligence technologies applied across various Civil Engineering sub-disciplines.

** Workflow Architecture:**
The pipeline consists of 4 main automated stages:

1.  **🕷️ Data Scraping:** Gathering raw data from RSS feeds, NewsAPI, and Google News.
2.  **🧠 NLP & Cleaning:** Content extraction (Selenium) and AI-powered summarization (Groq/Llama3).
3.  **🏷️ Classification:** Smart tagging using Dynamic Thresholds to categorize articles.
4.  **📊 Visualization:** Generating insights via Network Graphs, Heatmaps, and Word Clouds.

---
### 🛠️ Step 1: Environment Setup
The cell below imports all necessary Python libraries required for web scraping, data manipulation, Large Language Model (LLM) integration, and graphical plotting.

In [ ]:
# ==========================================
# 1. LIBRARIES (FOR THE ENTIRE PROJECT)
# ==========================================
import os
import sys
import time
import json
import re
import random
import itertools
from datetime import datetime
from collections import Counter
import warnings

# Data Processing
import pandas as pd
import numpy as np

# Natural Language Processing (NLP) & Machine Learning [UPDATED]
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer

# Web Scraping & Browser Automation
import requests
import feedparser
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from newspaper import Article

# Artificial Intelligence (AI)
from groq import Groq

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
from wordcloud import WordCloud, STOPWORDS

# Enable inline plotting for Jupyter Notebook
%matplotlib inline

# ==========================================
# SYSTEM CONFIGURATION & DOWNLOADS
# ==========================================

# 1. Suppress Warnings (To keep the notebook output clean)
warnings.filterwarnings("ignore")

# 2. NLTK Resource Check (Downloads automatically if missing)
print("⏳ Checking NLTK resources...")
try:
    nltk.data.find('tokenizers/punkt')
    nltk.data.find('corpora/stopwords')
    nltk.data.find('corpora/wordnet')
    print("   - NLTK resources found.")
except LookupError:
    print("   - Downloading missing NLTK resources...")
    nltk.download('punkt', quiet=True)
    nltk.download('stopwords', quiet=True)
    nltk.download('wordnet', quiet=True)
    print("   - Download complete.")

print("-" * 40)
print("✅ ALL LIBRARIES INSTALLED AND LOADED SUCCESSFULLY!")

## ⚙️ Step 2: Global Configurations & API Setup

**⚠️ Action Required:** Please input your personal API keys in the variables below before proceeding.

This cell acts as the **Central Control Unit** for the notebook. It defines:
1.  **API Credentials:** Securely stores access keys for **NewsAPI** (Data Scraping) and **Groq** (LLM Processing).
2.  **File Registry:** Standardizes input/output filenames to ensure seamless data flow between sequential tasks (e.g., Output of Task 1 -> Input of Task 2).

In [ ]:
# ==========================================
# 2. GLOBAL SETTINGS & API KEYS
# ==========================================

# 🔑 PASTE YOUR API KEYS HERE (Inside the quotes)
# -----------------------------------------------------------
NEWS_API_KEY = "PASTE_YOUR_NEWS_API_KEY_HERE" 
GROQ_API_KEY = "PASTE_YOUR_GROQ_API_KEY_HERE"
GROQ_API_KEY_Ber = "PASTE_YOUR_GROQ_API_KEY_HERE"
# -----------------------------------------------------------

# 📂 FILE NAMES (Constants for Automation)
# Defining filenames here ensures consistency across all Tasks.
FILE_RAW        = "CE49X_RAW_DATA_1000_FULL.xlsx"            # Task 1A Output (Scraping)
FILE_WITH_TEXT  = "CE49X_FINAL_DATASET_WITH_TEXT.xlsx"       # Task 1B Output (Full Text)
FILE_CLEAN_TEXT = "CE49X_TASK1_FINAL_CLEAN.xlsx"           # Task 1C Output (Pre-Cleaning)
FILE_AI_SUMMARY = "CE49X_TASK2_FINAL_SUBMISSIONBRT.xlsx" # Task 2A Output (AI Summary & TF-IDF )
FILE_NLP_FINAL  = "CE49X_Final_Clean_Related_Articles.xlsx"  # Task 2B Output (AI Filtered)
FILE_NGRAM_REPORT = "CE49X_Project_Top20Words_Top20bigrams_Top20Trigrams.xlsx" # Task 2C Output (N-grams) 
FILE_TAGGED     = "CE49X_Task3_Classification of Diciplines.xlsx" # Task 3 Output (Tagged)


# CONFIGURATION CHECK (Validates your input)
print(f"⚙️  Settings Loaded.")
print("-" * 30)

# Check if the user is still using the placeholder "PASTE" or left it empty
if "PASTE" in NEWS_API_KEY or len(NEWS_API_KEY) < 10:
    print(f"❌ WARNING: NewsAPI Key is missing or invalid!")
else:
    print(f"✅ NewsAPI Key: Detected ({NEWS_API_KEY[:4]}***)")

if "PASTE" in GROQ_API_KEY or len(GROQ_API_KEY) < 10:
    print(f"❌ WARNING: Groq API Key is missing or invalid!")
else:
    print(f"✅ Groq API Key: Detected ({GROQ_API_KEY[:4]}***)")

# 🕷️ Task 1A: Automated Data Scraping

This section initiates the data pipeline by collecting raw information from the web.

### 📋 Data Overview
We have successfully compiled a dataset containing **965 unique entries** related to Civil Engineering and Artificial Intelligence.

| **Attribute** | **Description** |
|:-------------|:----------------|
| **Sources** | RSS Feeds, Google News, NewsAPI |
| **Volume** | 965 Raw Articles |
| **Condition** | Raw (Contains noise, duplicates, and non-relevant entries) |
| **Output** | 'CE49X_RAW_DATA_1000_FULL.xlsx' |

### 🏗️ Data Schema
The collected data is structured into the following columns:
1.  **Title:** The main heading of the news or article.
2.  **Date:** When the content was published.
3.  **Source:** The publisher website.
4.  **URL:** Link to the full text.
5.  **Type:** The category of the source content.

*⚠️ **System Check:** The code below checks for existing data files to prevent redundant scraping.*

In [ ]:
# ==========================================
# TASK 1A: DATA SCRAPING (Dependent on Global Settings)
# ==========================================

# SAFETY CHECK: Has Cell 2 (Settings) been run?
if 'NEWS_API_KEY' not in globals() or 'FILE_RAW' not in globals():
    print("❌ ERROR: Please run the top 'Cell 2' (Global Settings) first!")
else:
    # Assign the global filename to a local variable for easier access
    RAW_DATA_FILE = FILE_RAW 

    # 1. Check for Existing File
    if os.path.exists(RAW_DATA_FILE):
        print(f"✅ '{RAW_DATA_FILE}' already exists. Skipping scraping to save time/API limits.")
        df_raw = pd.read_excel(RAW_DATA_FILE)
        
        print(f"\n📊 CURRENT FILE STATUS:")
        print(f"   -> Total Rows: {len(df_raw)}")
        display(df_raw.head()) 

    else:
        print("🚀 File not found, starting INTERNET SCRAPING process...")
        print(f"🔑 Using API Key: {NEWS_API_KEY[:5]}*** (Loaded from Cell 2)")

        # --- SEARCH KEYWORDS ---
        civil_terms = ["Construction", "Structural Engineering", "Geotechnical", "Transportation", "Infrastructure", "Concrete", "Bridge", "Tunnel"]
        ai_terms = ["Artificial Intelligence", "Machine Learning", "Computer Vision", "Generative AI", "Neural Networks", "Robotics", "Automation"]

        # --- 1. RSS FUNCTION (Google News) ---
        def fetch_rss_data():
            print(f"📡 Starting RSS Scraping...")
            articles = []
            # Create all combinations (e.g., Bridge + Robotics)
            combinations = list(itertools.product(civil_terms, ai_terms))
            
            # Google News RSS URL format (Targeting US/English)
            base_url = "https://news.google.com/rss/search?q={}+{}&hl=en-US&gl=US&ceid=US:en"

            for i, (civil, ai) in enumerate(combinations):
                if len(articles) >= 1200: break 
                
                query = f"{civil} {ai} when:2y"
                final_url = base_url.format(query.replace(" ", "+"), "")
                
                try:
                    feed = feedparser.parse(final_url)
                    for entry in feed.entries[:15]:
                        articles.append({
                            "Title": entry.title,
                            "Date": entry.published if 'published' in entry else datetime.now().strftime("%Y-%m-%d"),
                            "Source": entry.source.title if 'source' in entry else "Google News RSS",
                            "URL": entry.link,
                            "Type": "RSS (2-Year)"
                        })
                except Exception as e:
                    print(f"Error: {e}")
                
                # Polite delay to avoid IP ban
                time.sleep(random.uniform(0.5, 0.8)) 
            
            print(f"   -> RSS Completed. Total fetched: {len(articles)}")
            return articles

        # --- 2. API FUNCTION (NewsAPI) ---
        def fetch_api_data(api_key):
            # Check if the placeholder "PASTE" is still there or empty
            if not api_key or "PASTE" in api_key:
                print("⚠️ API Key is missing or invalid! Skipping API, using RSS data only.")
                return []
                
            print(f"\n🌍 Starting API Scraping (NewsAPI)...")
            articles = []
            queries = [
                "Civil Engineering Artificial Intelligence",
                "Construction Robotics Automation",
                "Structural Engineering Machine Learning",
                "Concrete Computer Vision",
                "Geotechnical Neural Networks",
                "Transportation Generative AI"
            ]
            url = "https://newsapi.org/v2/everything"
            
            for q in queries:
                if len(articles) >= 400: break
                
                params = {"q": q, "language": "en", "sortBy": "relevancy", "pageSize": 100, "apiKey": api_key}
                try:
                    response = requests.get(url, params=params)
                    data = response.json()
                    if data.get("status") == "ok":
                        items = data.get("articles", [])
                        for item in items:
                            articles.append({
                                "Title": item["title"],
                                "Date": item["publishedAt"],
                                "Source": item["source"]["name"],
                                "URL": item["url"],
                                "Type": "API (1-Month)"
                            })
                        print(f"   -> Query '{q}': {len(items)} articles found.")
                except Exception as e:
                    print(f"   -> API Connection Error: {e}")
            return articles

        # --- EXECUTION ---
        rss_data = fetch_rss_data()
        
        # Pass the global NEWS_API_KEY to the function
        api_data = fetch_api_data(NEWS_API_KEY) 

        # MERGE AND CLEAN
        all_data = rss_data + api_data
        df_raw = pd.DataFrame(all_data)

        print("\n🔍 Cleaning Data (Deduplication)...")
        initial_len = len(df_raw)
        
        # Remove duplicates based on URL
        df_raw = df_raw.drop_duplicates(subset=['URL'])
        
        # Remove duplicates based on Title (Case insensitive)
        df_raw['Title_Lower'] = df_raw['Title'].str.lower()
        df_raw = df_raw.drop_duplicates(subset=['Title_Lower'])
        df_raw = df_raw.drop(columns=['Title_Lower'])

        print(f"   -> {initial_len - len(df_raw)} duplicate records removed.")
        
        # SAVE
        df_raw.to_excel(RAW_DATA_FILE, index=False)
        print(f"✅ PROCESS COMPLETE! File saved: {RAW_DATA_FILE}")
        print(f"📊 Total Unique Articles: {len(df_raw)}")
        
        display(df_raw.head())

# Task 1B Content Extraction & Full-Text Scraping

## 🎯 Objective
Following the initial data collection, this module visits each unique URL to extract the **full textual content** of the articles. This step is crucial for transforming raw metadata (titles/links) into a rich corpus suitable for NLP analysis.

## ⚙️ Technical Approach
* **Selenium WebDriver:** Handles dynamic content loading and redirects (e.g., bypassing intermediate Google News links).
* **Newspaper3k Library:** Parses the HTML structure to isolate the main article text from ads, sidebars, and menus.
* **Error Handling:** Implements a "Force Restart" mechanism to recover from browser crashes or connection timeouts automatically.

## 📊 Process
1.  **Input:** `CE49X_RAW_DATA_1000_FULL.xlsx` (List of URLs).
2.  **Action:** The script navigates to ~965 URLs, handles redirects, and scrapes the body text.
3.  **Output:** `CE49X_FINAL_DATASET_WITH_TEXT.xlsx` (Dataset enriched with a `Full_Text` column).

In [ ]:
# ==========================================
# TASK 1B: CONTENT EXTRACTION (Selenium & Newspaper3k)
# ==========================================

# CHECK: Are global settings loaded?
if 'FILE_RAW' not in globals() or 'FILE_WITH_TEXT' not in globals():
    print("❌ ERROR: Please run 'Cell 2' (Global Settings) first!")
    INPUT_FILE = "CE49X_RAW_DATA_1000_FULL.xlsx"       # Fallback
    OUTPUT_FILE = "CE49X_FINAL_DATASET_WITH_TEXT.xlsx"  # Fallback
else:
    INPUT_FILE = FILE_RAW          # Input: Output of Task 1
    OUTPUT_FILE = FILE_WITH_TEXT   # Output: This task's result

# ==========================================
# FILE EXISTENCE CHECK (LIKE TASK 1A)
# ==========================================
if os.path.exists(OUTPUT_FILE):
    print(f"✅ '{OUTPUT_FILE}' already exists. Skipping scraping to save time.")
    df_result = pd.read_excel(OUTPUT_FILE)
    
    print(f"\n📊 CURRENT FILE STATUS:")
    print(f"   -> Total Rows: {len(df_result)}")
    print(f"   -> Filled Texts: {df_result['Full_Text'].notna().sum()}")
    display(df_result.head())

else:
    print(f"🚀 Output file not found. Starting CONTENT EXTRACTION process...")

# ==========================================
# BROWSER MANAGEMENT
# ==========================================
    def setup_driver():
        chrome_options = Options()
        chrome_options.add_argument("--no-sandbox")
        chrome_options.add_argument("--disable-dev-shm-usage")
        chrome_options.page_load_strategy = 'eager'
        
        # Disable logs to prevent console clutter
        chrome_options.add_argument("--log-level=3") 
        
        service = Service(ChromeDriverManager().install())
        try:
            driver = webdriver.Chrome(service=service, options=chrome_options)
            driver.set_page_load_timeout(20)
            return driver
        except Exception as e:
            print(f"🚨 Driver installation failed, retrying: {e}")
            time.sleep(5)
            return webdriver.Chrome(service=service, options=chrome_options)

    def force_restart_driver(driver):
        """Forcibly closes and restarts the browser if it crashes."""
        print("\n♻️  EMERGENCY: Restarting browser...")
        try:
            driver.quit()
        except:
            pass # Ignore errors if already closed
        time.sleep(2)
        return setup_driver()

    # ==========================================
    # CONTENT EXTRACTION LOGIC
    # ==========================================
    def get_full_content(driver, url):
        try:
            driver.get(url)
            time.sleep(3)
            
            # Handle Google redirects
            current_url = driver.current_url
            if "google.com" in current_url:
                time.sleep(2)
                current_url = driver.current_url

            article = Article(current_url)
            article.download()
            article.parse()
            
            if len(article.text) < 250:
                return current_url, article.text, "Short Content"
                
            return current_url, article.text, "Success"

        except Exception as e:
            # Return error as string to handle in main loop
            return url, "", str(e)

    # ==========================================
    # MAIN FLOW
    # ==========================================
    print("📂 Checking files...")

    if os.path.exists(OUTPUT_FILE):
        print(f"✅ Resuming from existing file: {OUTPUT_FILE}")
        df = pd.read_excel(OUTPUT_FILE)
    else:
        print(f"🆕 Starting from scratch using: {INPUT_FILE}")
        if os.path.exists(INPUT_FILE):
            df = pd.read_excel(INPUT_FILE)
        else:
            print(f"❌ ERROR: Input file '{INPUT_FILE}' not found. Run Task 1 first.")
            df = pd.DataFrame() # Create empty to prevent crash

    if not df.empty:
        if 'Full_Text' not in df.columns:
            df['Full_Text'] = ""
            df['Final_URL'] = ""
            df['Scrape_Status'] = ""

        print(f"🌐 Starting Browser... (Scanning remaining rows)")
        driver = setup_driver()

        save_interval = 5

        for index, row in df.iterrows():
            
            # 1. Skip if already processed
            if pd.notna(row['Full_Text']) and len(str(row['Full_Text'])) > 50:
                continue

            # 2. Periodic Cleanup (Every 50 steps)
            if index > 0 and index % 50 == 0:
                driver = force_restart_driver(driver)

            # 3. Start Processing
            original_url = row['URL']
            print(f"[{index+1}/{len(df)}] ⏳ Processing: {str(row['Title'])[:30]}...")
            
            real_url, text, status = get_full_content(driver, original_url)
            
            # === CRITICAL UPDATE: CRASH CONTROL ===
            # Check for connection errors or browser crashes
            if "HTTPConnectionPool" in status or "chrome not reachable" in status.lower() or "refused" in status.lower():
                print(f"   ❌ Browser CRASHED! ({status[:30]}...)")
                
                # Restart driver immediately
                driver = force_restart_driver(driver)
                
                # Retry the SAME link
                print("   🔄 Retrying same link...")
                real_url, text, status = get_full_content(driver, original_url)
            # ==========================================

            # Save Results
            df.at[index, 'Final_URL'] = real_url
            df.at[index, 'Full_Text'] = text
            df.at[index, 'Scrape_Status'] = status
            
            if status == "Success":
                print(f"   ✅ OK | {len(str(text))} Chars")
            else:
                print(f"   ⚠️ {status[:50]}") 
                
            if (index + 1) % save_interval == 0:
                df.to_excel(OUTPUT_FILE, index=False)
                print(f"   💾 Auto-saved.")

        # Cleanup
        try:
            driver.quit()
        except:
            pass

        df.to_excel(OUTPUT_FILE, index=False)
        print("\n🏁 PROCESS COMPLETED!")

# 🧹 Task 1C: Data Sanitation & Pre-Filtering

Before feeding the data into the NLP pipeline (Task 2), we must perform a **Quality Assurance (QA)** check. Sending broken or empty articles to the LLM (Large Language Model) would waste computational resources and API limits.

### 🔍 Process Logic
This script filters the dataset based on strict criteria:
1.  **Status Check:** Retains only rows marked as `Success` during scraping.
2.  **Length Validation:** Removes texts shorter than **200 characters** (often indicates captcha errors or empty pages).
3.  **Deduplication:** Ensures every article is unique by checking `URL` and `Title`.

* **Input:** Raw text data with potential errors.
* **Output:** A clean, high-quality corpus ready for AI processing (`CE49X_TASK1_FINAL_CLEAN.xlsx`).

In [ ]:
# ==========================================
# TASK 1C: DATA SANITATION (PRE-CLEANING)
# ==========================================
import pandas as pd
import os

# CHECK: Are global settings loaded?
if 'FILE_WITH_TEXT' not in globals() or 'FILE_CLEAN_TEXT' not in globals():
    print("❌ ERROR: Please run 'Cell 2' (Global Settings) first!")
    # Fallback to defaults if Cell 2 wasn't run
    INPUT_FILE = "CE49X_FINAL_DATASET_WITH_TEXT.xlsx"
    OUTPUT_FILE = "CE49X_TASK1_FINAL_CLEAN.xlsx"
else:
    # Use global variables defined in Cell 2
    INPUT_FILE = FILE_WITH_TEXT
    OUTPUT_FILE = FILE_CLEAN_TEXT

print(f"📂 Loading dataset: {INPUT_FILE} ...")

if not os.path.exists(INPUT_FILE):
    print(f"❌ ERROR: File '{INPUT_FILE}' not found. Please run Task 1.5 first.")
else:
    df = pd.read_excel(INPUT_FILE)
    initial_count = len(df)

    # STEP 1: Filter out failed scrapes or empty text
    # Keep only rows where Scrape_Status is 'Success' AND Full_Text is not empty
    df_cleaned = df[
        (df['Scrape_Status'] == 'Success') & 
        (df['Full_Text'].notna()) & 
        (df['Full_Text'].str.len() > 200) # Remove very short (likely error) texts
    ].copy()

    # STEP 2: Remove duplicate articles (Based on Title or URL)
    # Project Requirement: "Unique articles only"
    df_cleaned = df_cleaned.drop_duplicates(subset=['URL'])
    df_cleaned = df_cleaned.drop_duplicates(subset=['Title'])

    final_count = len(df_cleaned)

    # STEP 3: Save the clean file
    df_cleaned.to_excel(OUTPUT_FILE, index=False)

    print("\n" + "="*40)
    print(f"✅ SANITATION COMPLETED!")
    print(f"📥 Total Input Rows      : {initial_count}")
    print(f"📤 Successful Rows       : {final_count}")
    print(f"🗑️ Removed (Trash/Fail)  : {initial_count - final_count}")
    print(f"💾 New File Saved As     : {OUTPUT_FILE}")
    print("="*40)

    if final_count >= 500:
        print(f"🚀 CONGRATULATIONS! You have {final_count} articles (Target: 500+ Passed).")
    else:
        print(f"⚠️ WARNING: You have {final_count} articles remaining. You might need more data.")

# 🏗️ Task 2A Civil Engineering News Analysis: NLP Pipeline & AI Summarization

This notebook implements an automated Natural Language Processing (NLP) pipeline to analyze and summarize Civil Engineering news articles. The workflow integrates traditional NLP techniques with Generative AI to process raw text data into structured insights.

### 🚀 Key Features & Methodology

**1. Data Preprocessing (NLTK)**
* **Cleaning:** Removal of punctuation, numbers, and short words.
* **Normalization:** Lowercasing and tokenization.
* **Filtering:** Removal of standard English stopwords and domain-specific noise words (e.g., "subscribe", "advertisement").
* **Lemmatization:** Converting words to their base root form using `WordNetLemmatizer`.

**2. Generative AI Summarization (Groq API)**
* Utilizes the **Llama-3.1-8b** model via the Groq client.
* **Prompt Engineering:** Generates concise summaries (max 23 words) that *must* explicitly classify the news into a specific construction field (e.g., Structural, Geotechnical).

**3. Feature Extraction (TF-IDF)**
* Calculates **Term Frequency-Inverse Document Frequency** scores to identify the most significant keywords for each individual article, highlighting unique topics.

**5. Output**
* The processed data, including AI summaries and TF-IDF keywords, is exported to `CE49X_TASK2_FINAL_SUBMISSIONBRT.xlsx`.

In [ ]:
# ==========================================
# 3. TASK 2A: AI SUMMARIZATION & TF-IDF
# ==========================================

# 1. SETTINGS & FILE NAMES
# ------------------------------------------
# Check if global settings are loaded from Cell 2
if 'FILE_CLEAN_TEXT' not in globals() or 'FILE_AI_SUMMARY' not in globals():
    print("❌ ERROR: Please run 'Cell 2' (Global Settings) first!")
    INPUT_FILE_PATH = "CE49X_TASK1_FINAL_CLEAN.xlsx"
    OUTPUT_FILE_PATH = "CE49X_TASK2_FINAL_SUBMISSIONBRT.xlsx"
else:
    INPUT_FILE_PATH = FILE_CLEAN_TEXT  
    OUTPUT_FILE_PATH = FILE_AI_SUMMARY 

# ==========================================
# 2. FILE EXISTENCE CHECK (Save API Quota)
# ==========================================
if os.path.exists(OUTPUT_FILE_PATH):
    # If the file exists, skip the process
    print(f"✅ '{OUTPUT_FILE_PATH}' already exists. Skipping AI Analysis to save API quota.")
    print("   (Data is loaded and ready for next steps.)")
    df = pd.read_excel(OUTPUT_FILE_PATH)
    display(df.head())

else:
    # If file is MISSING, start the process (All logic is inside this block)
    print("🟢 File not found. Starting AI Analysis & NLP Pipeline...")

    # API Key Check
    if "PASTE" in GROQ_API_KEY_Ber or not GROQ_API_KEY_Ber:
        print("❌ ERROR: Please define your GROQ_API_KEY_Ber in the Global Settings cell!")
    elif not os.path.exists(INPUT_FILE_PATH):
        print(f"❌ ERROR: Input file '{INPUT_FILE_PATH}' not found. Please run Task 1C first.")
    else:
        # ------------------------------------------
        # AI & NLP CONFIGURATION
        # ------------------------------------------
        # Using Groq API for processing
        client = Groq(api_key=GROQ_API_KEY_Ber)
        MODEL_NAME = "llama-3.1-8b-instant"

        # NLP Setup
        # NLTK lemmatizer and stop words configuration
        lemmatizer = WordNetLemmatizer()
        custom_stop_words = set(stopwords.words('english'))
        custom_stop_words.update(['subscribe', 'click', 'here', 'read', 'more', 'advertisement', 'copyright', 'share', 'civil', 'engineering']) 

        def clean_pipeline(text):
            if not isinstance(text, str) or len(text) < 10: return ""
            text = re.sub(r'[^\w\s]', '', text.lower())
            text = re.sub(r'\d+', '', text)
            tokens = nltk.word_tokenize(text)
            cleaned = [lemmatizer.lemmatize(w) for w in tokens if w not in custom_stop_words and len(w) > 2]
            return " ".join(cleaned)

        def get_summary_groq(text):
            try:
                time.sleep(1.2) # Rate limit precaution
                prompt = (
                    f"Summarize this Civil Engineering news in 1 concise sentence (max 23 words). "
                    f"You MUST explicitly mention which specific construction field "
                    f"(e.g., Structural, Geotechnical, Materials, Transport) is involved: {text[:2500]}"
                )
                chat_completion = client.chat.completions.create(
                    messages=[{"role": "user", "content": prompt}],
                    model=MODEL_NAME, temperature=0.5,
                )
                return chat_completion.choices[0].message.content.strip()
            except Exception as e:
                if "429" in str(e):
                    time.sleep(10) # Wait if rate limited
                    return get_summary_groq(text)
                return "Summary Error"

        # Read Data & Process
        try:
            if INPUT_FILE_PATH.endswith('.csv'): df = pd.read_csv(INPUT_FILE_PATH)
            else: df = pd.read_excel(INPUT_FILE_PATH)
            
            summaries = []
            cleaned_texts = []
            print(f"🚀 Processing {len(df)} articles...")

            for index, row in df.iterrows():
                if (index + 1) % 10 == 0: print(f"[{index+1}/{len(df)}] Processing...")
                raw = str(row['Full_Text'])
                summaries.append(get_summary_groq(raw))
                cleaned_texts.append(clean_pipeline(raw))

            df['AI_Summary'] = summaries
            df['Cleaned_Text_NLP'] = cleaned_texts

            # TF-IDF Calculation
            print("\n📊 Calculating TF-IDF Scores...")
            vectorizer = TfidfVectorizer(max_features=1000)
            tfidf_matrix = vectorizer.fit_transform(df['Cleaned_Text_NLP'].fillna(""))
            feature_names = vectorizer.get_feature_names_out()
            top_keywords = []
            for i in range(tfidf_matrix.shape[0]):
                row = tfidf_matrix[i]
                if row.nnz > 0:
                    top_indices = row.toarray()[0].argsort()[-5:][::-1]
                    keywords = [feature_names[idx] for idx in top_indices if row[0, idx] > 0]
                    top_keywords.append(", ".join(keywords))
                else: top_keywords.append("")
            df['Top_TFIDF_Keywords'] = top_keywords

            # Save to File
            df.to_excel(OUTPUT_FILE_PATH, index=False)
            print(f"✅ NEW ANALYSIS SAVED: {OUTPUT_FILE_PATH}")

        except Exception as e:
            print(f"❌ ERROR: {e}")

# 🛡️ Task 2B: AI Relevance Filtering 

**NOTE:** This step uses the dataset (`CE49X_TASK2_FINAL_SUBMISSIONBRT.xlsx`).

## 🎯 Objective
To perform a semantic quality check on the provided dataset using the **Groq API (Llama 3)**. The AI acts as a Civil Engineering Professor to filter out irrelevant content.

## ⚙️ Process
1.  **Input:** External File defined in Cell 2 (`FILE_AI_SUMMARY`).
2.  **Action:** The AI reads the Title/Summary and decides: **Keep** or **Reject**.
3.  **Output:** Saves to the standard pipeline filename (`CE49X_Final_Clean_Related_Articles.xlsx`) so Task 3 can read it.

In [ ]:

# ==========================================
# TASK 2B: AI RELEVANCE FILTERING (EXTERNAL INPUT)
# ==========================================
import pandas as pd
from groq import Groq
import time
import json
import re
import os

# ==========================================
# 1. SETTINGS & SETUP
# ==========================================

# CHECK: Are global settings (Cell 2) loaded?
if 'GROQ_API_KEY' not in globals() or 'FILE_EXTERNAL_INPUT' not in globals():
    print("❌ ERROR: Please run 'Cell 2' (Global Settings) first!")
    # Fallback variables just in case
    INPUT_FILE = "CE49X_TASK2_FINAL_SUBMISSIONBRT.xlsx" 
    FINAL_FILE = "CE49X_Final_Clean_Related_Articles.xlsx"
    CURRENT_API_KEY = "" 
else:
    # Use Global Variables
    INPUT_FILE = FILE_AI_SUMMARY
    FINAL_FILE = FILE_NLP_FINAL
    CURRENT_API_KEY = GROQ_API_KEY_Ber

# Temporary Files
TEMP_FILE = "TEMP_ALL_DATA_BACKUP.xlsx"
LOG_FILE = "completed_indices.txt"

# Initialize Client
if not CURRENT_API_KEY or "PASTE" in CURRENT_API_KEY:
    print("❌ ERROR: Groq API Key is missing! Check Cell 2.")
    client = None
else:
    client = Groq(api_key=CURRENT_API_KEY)

# ==========================================
# 2. FILE LOADING
# ==========================================
print(f"📂 Reading file: {INPUT_FILE}")

def load_file(path):
    try: return pd.read_csv(path)
    except:
        try: return pd.read_csv(path, sep=';')
        except: return pd.read_excel(path)

if os.path.exists(FINAL_FILE):
    print(f"✅ '{FINAL_FILE}' already exists. Skipping AI Filtering.")
    df = pd.read_excel(FINAL_FILE) # Load existing for display if needed
    print(f"💎 Clean Data Count: {len(df)}")
else:
    if not os.path.exists(INPUT_FILE):
        print(f"❌ ERROR: Input file '{INPUT_FILE}' not found!")
        df = pd.DataFrame()
    else:
        try:
            df = load_file(INPUT_FILE)
            print(f"✅ File loaded. Initial Row Count: {len(df)}")
        except Exception as e:
            print(f"❌ ERROR: Could not read file. ({e})")
            df = pd.DataFrame()

# Proceed only if client exists and data is loaded (and output doesn't exist)
if client and not df.empty and not os.path.exists(FINAL_FILE):

    # Column Mapping
    text_col = 'AI_Summary' if 'AI_Summary' in df.columns else 'Full_Text'
    # Fallback if specific columns don't exist
    if text_col not in df.columns: text_col = df.columns[1]

    title_col = 'Title' if 'Title' in df.columns else df.columns[0]

    # Initialize Status Columns
    if 'is_civil_ai' not in df.columns: df['is_civil_ai'] = False
    if 'Status_Detail' not in df.columns: df['Status_Detail'] = "Pending"

    print(f"ℹ️  Using columns -> Title: '{title_col}', Text: '{text_col}'")

    # ==========================================
    # 3. SAFE API FUNCTION
    # ==========================================
    def ask_groq_safe(prompt):
        wait_time = 20
        while True:
            try:
                completion = client.chat.completions.create(
                    model="llama-3.3-70b-versatile",
                    messages=[{"role": "user", "content": prompt}],
                    temperature=0,
                    max_tokens=150,
                )
                return completion.choices[0].message.content.strip()
            except Exception as e:
                error_msg = str(e).lower()
                if "429" in error_msg or "rate limit" in error_msg:
                    print(f"\n⚠️ RATE LIMIT (429). Waiting {wait_time}s...")
                    time.sleep(wait_time)
                    wait_time += 10
                else:
                    print(f"\n❌ Error: {e}. Retrying in 5s.")
                    time.sleep(5)

    # ==========================================
    # 4. RESUME CAPABILITY
    # ==========================================
    completed_indices = set()
    if os.path.exists(LOG_FILE):
        with open(LOG_FILE, 'r') as f:
            for line in f:
                try: completed_indices.add(int(line.strip()))
                except: pass

    todo_indices = [i for i in df.index if i not in completed_indices]
    BATCH_SIZE = 10 
    total_steps = (len(todo_indices) // BATCH_SIZE) + 1

    print(f"⏭️  Already Completed: {len(completed_indices)}")
    print(f"🚀  Remaining to Process: {len(todo_indices)}")
    print("-" * 60)

    # ==========================================
    # 5. MAIN LOOP
    # ==========================================
    step = 0
    for i in range(0, len(todo_indices), BATCH_SIZE):
        step += 1
        batch_indices = todo_indices[i : i+BATCH_SIZE]
        batch_df = df.loc[batch_indices]
        
        batch_text = ""
        for idx, row in batch_df.iterrows():
            summary = str(row[text_col])[:700].replace("\n", " ") 
            title = str(row[title_col])[:150]
            batch_text += f"Paper_ID {idx}: [TITLE: {title}] [SUMMARY: {summary}]\n\n"

        # --- PROMPT ---
        prompt = f"""
        Act as a Civil Engineering Professor.
        Filter papers to keep ONLY those relevant to Civil Engineering, Construction, and the Built Environment.
        
        CRITERIA FOR INCLUSION (YES - KEEP):
        1. Core Civil: Structures, Geotechnics, Transport, Materials, Hydraulics.
        2. Construction: Management, Safety, BIM, Digital Twins, Heavy Equipment.
        3. Related Built Environment: Architecture, Urban Planning, Smart Cities.
        4. AI Applications: ANY AI/ML paper applied to the domains above.
        
        CRITERIA FOR EXCLUSION (NO - REJECT):
        1. Pure Non-Civil Fields: Medical, Finance, Pure Biology, Agriculture.
        2. General CS: Pure algorithms, NLP, or Gaming without a construction use case.
        
        DECISION RULE:
        If a paper is borderline, INCLUDE IT.
        
        INPUT LIST:
        {batch_text}
        
        OUTPUT FORMAT:
        Return ONLY a JSON list of ACCEPTED Paper_ID integers. Example: [102, 105]
        If none are relevant, return [].
        """
        
        print(f"\n🔄 Processing Batch {step}/{total_steps}...", end="")
        response = ask_groq_safe(prompt)
        
        accepted_ids = []
        try:
            match = re.search(r'\[.*?\]', response)
            if match:
                accepted_ids = json.loads(match.group(0))
        except: pass

        print(f" (✅ {len(accepted_ids)} Accepted)")

        for idx in batch_indices:
            title_short = str(df.loc[idx, title_col])[:40] + "..."
            if idx in accepted_ids:
                print(f"  🟢 [KEEP] ID {idx}: {title_short}")
                df.loc[idx, 'is_civil_ai'] = True
                df.loc[idx, 'Status_Detail'] = "ACCEPTED"
            else:
                print(f"  🔴 [DROP] ID {idx}: {title_short}")
                df.loc[idx, 'is_civil_ai'] = False
                df.loc[idx, 'Status_Detail'] = "REJECTED"

        # Save Progress
        with open(LOG_FILE, 'a') as f:
            for x in batch_indices:
                f.write(f"{x}\n")
        
        try: df.to_excel(TEMP_FILE, index=False)
        except: pass
            
        time.sleep(1)

    # ==========================================
    # 6. FINAL CLEANUP
    # ==========================================
    print("\n" + "="*60)
    print("🧹 FINALIZING: Removing rejected articles...")

    df_clean = df[df['is_civil_ai'] == True].copy()
    cols_to_drop = ['is_civil_ai', 'Status_Detail']
    df_clean.drop(columns=[c for c in cols_to_drop if c in df_clean.columns], inplace=True)

    df_clean.to_excel(FINAL_FILE, index=False)

    print(f"✅ PROCESS COMPLETED!")
    print(f"📄 Original Data Count: {len(df)}")
    print(f"💎 Final Clean Data Count: {len(df_clean)}")
    print(f"💾 FILE READY: {FINAL_FILE}")
    print("="*60)

# 📊 Task 2C: Advanced Keyword Analysis (N-Grams)

## 🎯 Objective
To extract the most frequent and meaningful terminology from the dataset, we perform an **N-Gram Analysis**. This goes beyond simple word counting by identifying common phrases and technical terms.

## 🧹 Noise Filtering Strategy
To ensure the results are relevant to Engineering, we apply a **Dual-Layer Stop Word Filter**:
1.  **Hard Filter (Unigrams):** Removes generic terms like "system," "model," "data," and "analysis" to reveal specific topics (e.g., "Concrete," "Seismic").
2.  **Soft Filter (Bi/Trigrams):** Allows generic terms only when they form technical phrases (e.g., "Machine Learning," "Data Analysis," "Structural Health Monitoring").

## 📈 Output
* **Unigrams:** Top single keywords (Topic indicators).
* **Bigrams:** Top 2-word phrases (Technology pairs).
* **Trigrams:** Top 3-word phrases (Specific methodologies).
* **Excel Report:** `CE49X_Project_Top20Words_Top20bigrams_Top20Trigrams.xlsx`

In [ ]:

# ==========================================
# TASK 2C: ADVANCED KEYWORD ANALYSIS (N-GRAMS)
# ==========================================
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, ENGLISH_STOP_WORDS
import os

# 1. CHECK GLOBAL SETTINGS
# ---------------------------------------------------------
if 'FILE_NLP_FINAL' not in globals() or 'FILE_NGRAM_REPORT' not in globals():
    print("❌ ERROR: Please run 'Cell 2' (Global Settings) first!")
    # Fallback defaults
    INPUT_FILE = "CE49X_Final_Clean_Related_Articles.xlsx"
    OUTPUT_FILE = "CE49X_Project_Top20Words_Top20bigrams_Top20Trigrams.xlsx"
else:
    # Use variables from Cell 2
    INPUT_FILE = FILE_NLP_FINAL      # Input: Clean Data from Task 2A
    OUTPUT_FILE = FILE_NGRAM_REPORT  # Output: The report file

print(f"📊 Reading dataset: {INPUT_FILE}")

if not os.path.exists(INPUT_FILE):
    print(f"❌ ERROR: File '{INPUT_FILE}' not found! Run Task 2A first.")
else:
    df = pd.read_excel(INPUT_FILE)

    # ==========================================
    # 2. TEXT MERGING (FULL SCAN)
    # ==========================================
    # We combine available text columns for maximum context
    col1 = 'Cleaned_Text_NLP' if 'Cleaned_Text_NLP' in df.columns else ''
    col2 = 'AI_Summary' if 'AI_Summary' in df.columns else ''
    
    # Fallback to Full_Text or Title if columns are missing
    if not col1 and not col2:
        col1 = 'Full_Text' if 'Full_Text' in df.columns else 'Title'

    print(f"🔍 Analyzing Columns: {col1} + {col2}")

    df['Combined_Text'] = (df[col1].fillna("").astype(str) + " " + df[col2].fillna("").astype(str)).str.lower()
    texts = df['Combined_Text'].tolist()

    # ==========================================
    # 3. STOP WORDS CONFIGURATION
    # ==========================================
    # Common junk words to exclude from all lists
    common_junk = [
        'google', 'scholar', 'crossref', 'text', 'full', 'view', 'download', 'pdf',
        'citation', 'cited', 'copyright', 'author', 'rights', 'reserved', 'license',
        'journal', 'publishing', 'publisher', 'volume', 'issue', 'peer', 'review',
        'online', 'library', 'access', 'open', 'website', 'page', 'web', 'homepage',
        'prediction', 'error', 'accuracy', 'mean', 'absolute', 'training', 'testing', 
        'validation', 'train', 'test', 'predict', 'predicted', 'values', 'value',
        'dataset', 'data', 'point', 'points', 'set', 'sets', 'center', 'centers',
        'proposed', 'approach', 'method', 'methodology', 'result', 'results',
        'performance', 'comparison', 'compared', 'experimental', 'study', 'studies',
        'using', 'used', 'use', 'based', 'table', 'figure', 'fig', 'doi', 'vol', 'no', 
        'pp', 'al', 'et', 'url', 'http', 'https', 'click', 'size', 
        'image', 'shown', 'presented', 'article', 'paper', 'work',
        'usd', 'billion', 'million', 'market', 'growth', 'report', 'forecast',
        'correlation', 'coefficient', 'input', 'feature', 'hidden', 'layer',
        'north', 'america', 'new', 'like', 'good', 'better', 'best', 'high', 'low', 
        'different', 'real', 'case', 'making', 'potential', 'need', 'needs', 
        'company', 'companies', 'parameter', 'parameters', 'information', 'technology',
        'significant', 'significantly', 'increase', 'decreased', 'impact'
    ]

    # Words banned only in Unigrams (Single words)
    single_word_bans = [
        'system', 'systems', 'model', 'models', 'analysis', 'algorithm', 'algorithms',
        'application', 'applications', 'process', 'project', 'design', 'development',
        'research', 'time', 'year', 'number', 'level', 'quality', 'state',
        'machine', 'learning', 'artificial', 'intelligence', 'deep', 'neural', 'network', 'networks', 
        'make', 'said', 'including', 'problem', 'solution', 'engineering', 'civil'
    ]

    stops_soft = list(ENGLISH_STOP_WORDS) + common_junk        
    stops_hard = stops_soft + single_word_bans                 

    # ==========================================
    # 4. ANALYSIS ENGINE
    # ==========================================
    def get_clean_top_n(corpus, n=20, n_gram=(1,1), custom_stops=None):
        if not corpus: return []
        try:
            vec = CountVectorizer(stop_words=custom_stops, ngram_range=n_gram, max_features=10000).fit(corpus)
            bag_of_words = vec.transform(corpus)
            sum_words = bag_of_words.sum(axis=0) 
            words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
            sorted_freq = sorted(words_freq, key = lambda x: x[1], reverse=True)
            
            clean_list = []
            for word, freq in sorted_freq:
                if " " in word:
                    parts = word.split()
                    if len(set(parts)) != len(parts): continue 
                clean_list.append((word, freq))
                if len(clean_list) == n: break
            return clean_list
        except ValueError:
            return []

    print("⏳ Running detailed N-Gram analysis...")

    # 1. UNIGRAMS
    top_words = get_clean_top_n(texts, 20, (1, 1), custom_stops=stops_hard)
    # 2. BIGRAMS
    top_bigrams = get_clean_top_n(texts, 20, (2, 2), custom_stops=stops_soft)
    # 3. TRIGRAMS
    top_trigrams = get_clean_top_n(texts, 20, (3, 3), custom_stops=stops_soft)

    # ==========================================
    # 5. DISPLAY AND SAVE
    # ==========================================
    def print_table(title, data):
        print("\n" + "="*50)
        print(f"🏆 {title}")
        print("="*50)
        for w, f in data:
            print(f"{w}: {f}")

    print_table("TOP 20 KEYWORDS (Unigrams)", top_words)
    print_table("TOP 20 PAIRS (Bigrams)", top_bigrams)
    print_table("TOP 20 TRIPLETS (Trigrams)", top_trigrams)

    # Save to Excel
    print(f"\n💾 Saving Report to: {OUTPUT_FILE}")
    with pd.ExcelWriter(OUTPUT_FILE, engine='openpyxl') as writer:
        pd.DataFrame(top_words, columns=['Word', 'Frequency']).to_excel(writer, sheet_name='Top20_Words', index=False)
        pd.DataFrame(top_bigrams, columns=['BiGram', 'Frequency']).to_excel(writer, sheet_name='Top20_Bigrams', index=False)
        pd.DataFrame(top_trigrams, columns=['TriGram', 'Frequency']).to_excel(writer, sheet_name='Top20_Trigrams', index=False)

    print(f"✅ ANALYSIS REPORT READY!")

# 🏷️ Task 3: Intelligent Classification & Visualization

## 🎯 Objective
This module categorizes the filtered articles into specific **Civil Engineering sub-domains** (e.g., *Structural, Geotechnical*) and **AI Technologies** (e.g., *Computer Vision, Predictive Analytics*).

## 🧠 Dynamic Thresholding Logic
To ensure high accuracy, we apply different keyword density thresholds based on our data analysis:
* **High Threshold (4):** Applied to broad categories like *Construction Management* and *Robotics* to reduce noise.
* **Low Threshold (2):** Applied to niche categories like *Geotechnical* and *Generative Design* to capture rare but relevant papers.

## 📊 Visual Outputs
1.  **Heatmap:** Visualizes the cross-disciplinary density (e.g., How often is *Computer Vision* used in *Structural Engineering*?).
2.  **Trend Graph:** Tracks the popularity of top AI+Civil combinations over time.

In [ ]:
# ==========================================
# TASK 3: CLASSIFICATION & VISUALIZATION
# ==========================================
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import os

# CHECK: Are global settings loaded?
if 'FILE_NLP_FINAL' not in globals() or 'FILE_TAGGED' not in globals():
    print("❌ ERROR: Please run 'Cell 2' (Global Settings) first!")
    INPUT_FILE = "CE49X_Final_Clean_Related_Articles.xlsx"
    OUTPUT_FILE = "CE49X_Task3_Classification of Diciplines.xlsx"
else:
    INPUT_FILE = FILE_NLP_FINAL   # Input: Output of Task 2A
    OUTPUT_FILE = FILE_TAGGED     # Output: This task's result

# IMAGE OUTPUTS
HEATMAP_FILE = "CE49X_Task3_Heatmap.png"
TREND_FILE = "CE49X_Task3_Trend_Analysis_COMBINATIONS.png"
DATE_COL = "Date"

print(f"📂 Reading File: {INPUT_FILE}")

if not os.path.exists(INPUT_FILE):
    print(f"❌ ERROR: File '{INPUT_FILE}' not found. Run Task 2.5 first.")
else:
    df = pd.read_excel(INPUT_FILE)

    # ==========================================
    # 1. TEXT PREPARATION
    # ==========================================
    # Combine relevant text columns for searching
    col1 = 'Cleaned_Text_NLP' if 'Cleaned_Text_NLP' in df.columns else ''
    col2 = 'AI_Summary' if 'AI_Summary' in df.columns else ''
    
    # Fallback if columns don't exist
    if not col1 and not col2: col1 = 'Full_Text'

    df['search_text'] = (df[col1].fillna("").astype(str) + " " + df[col2].fillna("").astype(str)).str.lower()

    # ==========================================
    # 2. KEYWORDS DICTIONARY
    # ==========================================
    ce_keywords = {
        "Structural": ["structural", "structure", "beam", "column", "concrete", "steel", "bridge", "seismic", "earthquake", "health monitoring", "shm", "crack", "damage", "masonry", "reinforced", "compressive", "tensile"],
        "Geotechnical": ["geotechnical", "soil", "rock", "foundation", "tunnel", "excavation", "slope", "stability", "landslide", "underground", "pile", "earth", "clay", "sand", "liquefaction"],
        "Transportation": ["transport", "transportation", "traffic", "road", "highway", "vehicle", "autonomous", "driverless", "logistics", "pavement", "asphalt", "flow", "pedestrian", "congestion"],
        "Construction Mgmt": ["management", "scheduling", "schedule", "cost", "estimation", "safety", "site", "worker", "risk", "bim", "building information", "planning", "contract", "supply chain"],
        "Environmental": ["environmental", "sustainability", "sustainable", "waste", "green", "energy", "carbon", "emission", "water", "pollution", "climate", "lifecycle", "lca"]
    }

    ai_keywords = {
        "Computer Vision": ["vision", "image", "camera", "video", "detection", "recognition", "cnn", "convolutional", "object detection", "segmentation", "drone", "uav", "surveillance"],
        "Predictive Analytics": ["prediction", "predictive", "forecast", "forecasting", "regression", "classification", "machine learning", "deep learning", "neural network", "ann", "lstm", "random forest", "svm", "risk assessment", "decision tree"],
        "Generative Design": ["generative", "optimization", "genetic algorithm", "evolutionary", "parametric", "topology", "design optimization"],
        "Robotics/Automation": ["robot", "robotics", "automation", "automated", "autonomous", "3d printing", "additive manufacturing", "sensor", "iot", "internet of things"]
    }

    # ==========================================
    # 3. DYNAMIC THRESHOLD LOGIC 🧠
    # ==========================================
    print("🏷️  Starting Classification (Using Optimized Thresholds)...")

    # --- CIVIL ENG THRESHOLDS ---
    CE_THRESHOLDS = {
        "Structural": 4,        
        "Transportation": 4,    
        "Construction Mgmt": 4, 
        "Geotechnical": 2,      
        "Environmental": 2      
    }

    # --- AI THRESHOLDS ---
    AI_THRESHOLDS = {
        "Robotics/Automation": 4,  # High noise -> High threshold
        "Predictive Analytics": 4, # High noise -> High threshold
        "Computer Vision": 3,      # Average -> Medium threshold
        "Generative Design": 2     # Niche -> Low threshold
    }

    def get_flexible_tags(text, keyword_dict, threshold_dict=None, default_threshold=2):
        scores = {category: 0 for category in keyword_dict}
        
        for category, words in keyword_dict.items():
            for word in words:
                scores[category] += text.count(word)
        
        winners = []
        
        for cat, score in scores.items():
            limit = threshold_dict.get(cat, default_threshold) if threshold_dict else default_threshold
            if score >= limit:
                winners.append(cat)
                
        # Recovery Mode: If no category passes the threshold, take the max score
        if not winners and max(scores.values()) > 0:
            max_val = max(scores.values())
            winners = [cat for cat, score in scores.items() if score == max_val]
            
        return winners if winners else ["Other"]

    # Apply Tagging
    df['CE_Area'] = df['search_text'].apply(lambda x: get_flexible_tags(x, ce_keywords, threshold_dict=CE_THRESHOLDS))
    df['AI_Tech'] = df['search_text'].apply(lambda x: get_flexible_tags(x, ai_keywords, threshold_dict=AI_THRESHOLDS))

    # Save to Excel
    df['CE_Area_Str'] = df['CE_Area'].apply(lambda x: ", ".join(x))
    df['AI_Tech_Str'] = df['AI_Tech'].apply(lambda x: ", ".join(x))

    url_col = 'Final_URL' if 'Final_URL' in df.columns else 'URL'
    # Rename Final_URL to URL for cleaner output
    df.rename(columns={url_col: 'URL'}, inplace=True)
    
    output_cols = ['Title', DATE_COL, 'CE_Area_Str', 'AI_Tech_Str', 'URL', 'search_text']
    df_save = df[[c for c in output_cols if c in df.columns]].copy()

    df_save.to_excel(OUTPUT_FILE, index=False)
    print(f"✅ Classifications saved to: {OUTPUT_FILE}")

    # ==========================================
    # 4. HEATMAP VISUALIZATION
    # ==========================================
    matrix_data = pd.DataFrame(0, index=ce_keywords.keys(), columns=ai_keywords.keys())

    for _, row in df.iterrows():
        ce_tags = row['CE_Area']
        ai_tags = row['AI_Tech']
        if "Other" in ce_tags or "Other" in ai_tags: continue
        for ce in ce_tags:
            for ai in ai_tags:
                matrix_data.loc[ce, ai] += 1

    plt.figure(figsize=(10, 6))
    sns.heatmap(matrix_data, annot=True, fmt="d", cmap="Blues", linewidths=.5)
    plt.title("Civil Engineering Areas vs. AI Technologies", fontsize=12)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig(HEATMAP_FILE, dpi=300)
    print(f"🖼️  Heatmap saved: {HEATMAP_FILE}")

    # ==========================================
    # 5. TREND ANALYSIS (TIME SERIES)
    # ==========================================
    print(f"\n📈 Generating Trend Analysis...")
    if DATE_COL in df.columns:
        try:
            # Convert Date
            df['Date_Object'] = pd.to_datetime(df[DATE_COL], errors='coerce', utc=True)
            df = df.dropna(subset=['Date_Object'])
            df['Month_Str'] = df['Date_Object'].dt.strftime('%Y-%m')
            
            # Explode lists to handle multiple tags
            df_exploded = df.explode('CE_Area').explode('AI_Tech')
            df_exploded = df_exploded[(df_exploded['CE_Area'] != 'Other') & (df_exploded['AI_Tech'] != 'Other')]
            
            # Create Combinations
            df_exploded['Combination'] = df_exploded['CE_Area'] + " + " + df_exploded['AI_Tech']
            
            # Find Top 5 Combinations
            top_combos = df_exploded['Combination'].value_counts().head(5).index.tolist()
            df_final_trend = df_exploded[df_exploded['Combination'].isin(top_combos)]
            
            # Pivot Data for Plotting
            trend_data = df_final_trend.groupby(['Month_Str', 'Combination']).size().unstack(fill_value=0)
            
            # Plot
            plt.figure(figsize=(14, 7))
            sns.lineplot(data=trend_data, marker="o", linewidth=2.5)
            plt.title("Top AI + Civil Engineering Research Trends Over Time", fontsize=14)
            plt.ylabel("Article Count")
            plt.xlabel("Month")
            plt.legend(title="Research Areas", bbox_to_anchor=(1.01, 1), loc='upper left')
            plt.xticks(rotation=45, ha='right')
            plt.grid(True, linestyle='--', alpha=0.7)
            plt.tight_layout()
            plt.savefig(TREND_FILE, dpi=300)
            print(f"📈 Trend Graph saved: {TREND_FILE}")
            
        except Exception as e:
            print(f"⚠️ Trend Analysis Error: {e}")

    print("\n✅ TASK 3 COMPLETED SUCCESSFULLY!")

# 📊 Task 4: Visualization Generation (Final Outputs)

## 🎯 Objective
This module transforms the analyzed data into publication-ready visual assets to demonstrate the relationships between Civil Engineering domains and AI technologies.

## 🖼️ Generated Visuals
1.  **Bar Chart:** Displays the distribution of articles across the 5 main Civil Engineering disciplines using dynamic thresholds.
2.  **Network Graph:** Maps the co-occurrence strength between specific engineering terms (e.g., *Seismic*) and AI methods (e.g., *Neural Networks*). Thicker lines indicate stronger connections.
3.  **Word Clouds:** Generates specific word clouds for each discipline, filtering out generic academic noise to highlight niche terminology.

## 📂 Output
All images are automatically saved to the folder: `Task4_Final_Outputs_Synchronized/`

In [ ]:
# ==========================================
# TASK 4: VISUALIZATION GENERATION
# ==========================================
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import networkx as nx
from wordcloud import WordCloud, STOPWORDS
import os
from collections import Counter
import itertools
import numpy as np

# CHECK: Are global settings loaded?
if 'FILE_NLP_FINAL' not in globals():
    print("❌ ERROR: Please run 'Cell 2' (Global Settings) first!")
    INPUT_FILE = "CE49X_Final_Clean_Related_Articles.xlsx"
else:
    INPUT_FILE = FILE_NLP_FINAL

# 🔥 AUTOMATIC FOLDER CREATION
base_dir = os.path.dirname(os.path.abspath(INPUT_FILE)) if os.path.exists(INPUT_FILE) else "."
OUTPUT_FOLDER = os.path.join(base_dir, "Task4_Final_Outputs_Synchronized")

if not os.path.exists(OUTPUT_FOLDER):
    os.makedirs(OUTPUT_FOLDER)

print(f"📂 Reading File: {INPUT_FILE}")
if not os.path.exists(INPUT_FILE):
    print("❌ ERROR: File not found! Run Task 2A first.")
else:
    df = pd.read_excel(INPUT_FILE)

    # Text Merging
    col1 = 'Cleaned_Text_NLP' if 'Cleaned_Text_NLP' in df.columns else ''
    col2 = 'AI_Summary' if 'AI_Summary' in df.columns else ''
    # Fallback to Full_Text if columns missing
    if not col1 and not col2: col1 = 'Full_Text'
    
    df['Combined_Text'] = (df[col1].fillna("").astype(str) + " " + df[col2].fillna("").astype(str)).str.lower()

    # ==========================================
    # 2. KEYWORDS DICTIONARY
    # ==========================================
    ce_keywords = {
        "Structural": ["structural", "structure", "beam", "column", "concrete", "steel", "bridge", "seismic", "earthquake", "health monitoring", "shm", "crack", "damage", "masonry", "reinforced", "compressive", "tensile"],
        "Geotechnical": ["geotechnical", "soil", "rock", "foundation", "tunnel", "excavation", "slope", "stability", "landslide", "underground", "pile", "earth", "clay", "sand", "liquefaction"],
        "Transportation": ["transport", "transportation", "traffic", "road", "highway", "vehicle", "autonomous", "driverless", "logistics", "pavement", "asphalt", "flow", "pedestrian", "congestion"],
        "Construction Mgmt": ["management", "scheduling", "schedule", "cost", "estimation", "safety", "site", "worker", "risk", "bim", "building information", "planning", "contract", "supply chain"],
        "Environmental": ["environmental", "sustainability", "sustainable", "waste", "green", "energy", "carbon", "emission", "water", "pollution", "climate", "lifecycle", "lca"]
    }

    ai_keywords = {
        "Computer Vision": ["vision", "image", "camera", "video", "detection", "recognition", "cnn", "convolutional", "object detection", "segmentation", "drone", "uav", "surveillance"],
        "Predictive Analytics": ["prediction", "predictive", "forecast", "forecasting", "regression", "classification", "machine learning", "deep learning", "neural network", "ann", "lstm", "random forest", "svm", "risk assessment", "decision tree"],
        "Generative Design": ["generative", "optimization", "genetic algorithm", "evolutionary", "parametric", "topology", "design optimization"],
        "Robotics/Automation": ["robot", "robotics", "automation", "automated", "autonomous", "3d printing", "additive manufacturing", "sensor", "iot", "internet of things"]
    }

    # ==========================================
    # 3. TAGGING FOR BAR CHART (DYNAMIC THRESHOLDS 🧠)
    # ==========================================
    print("🏷️  Preparing Bar Chart Data (Dynamic Thresholds)...")

    DYNAMIC_THRESHOLDS = {
        "Structural": 4,        
        "Transportation": 4,    
        "Construction Mgmt": 4, 
        "Geotechnical": 2,      
        "Environmental": 2      
    }

    def get_flexible_tags(text, keyword_dict, threshold_dict=None, default_threshold=2):
        scores = {category: 0 for category in keyword_dict}
        for category, words in keyword_dict.items():
            for word in words:
                scores[category] += text.count(word)
        
        winners = []
        for cat, score in scores.items():
            limit = threshold_dict.get(cat, default_threshold) if threshold_dict else default_threshold
            if score >= limit:
                winners.append(cat)
                
        # Recovery mechanism
        if not winners and max(scores.values()) > 0:
            max_val = max(scores.values())
            winners = [cat for cat, score in scores.items() if score == max_val]
            
        return winners

    # Multi-label tagging for Bar Chart
    df['CE_Area_Bar'] = df['Combined_Text'].apply(lambda x: get_flexible_tags(x, ce_keywords, threshold_dict=DYNAMIC_THRESHOLDS))

    # ==========================================
    # 4. VISUAL 1: BAR CHART
    # ==========================================
    print("📊 1. Generating Bar Chart...")
    df_exploded = df.explode('CE_Area_Bar')
    df_exploded = df_exploded[df_exploded['CE_Area_Bar'].notna()] 

    plt.figure(figsize=(10, 6))
    ax = sns.countplot(y="CE_Area_Bar", data=df_exploded, 
                       order=df_exploded['CE_Area_Bar'].value_counts().index, 
                       palette="viridis")
    plt.title("Number of Articles per Civil Engineering Area (Dynamic Thresholds)", fontsize=14)
    plt.xlabel("Article Count")
    plt.ylabel("CE Sub-Discipline")
    for container in ax.containers:
        ax.bar_label(container)
    plt.tight_layout()
    plt.savefig(f"{OUTPUT_FOLDER}/1_BarChart_Count_SYNC.png", dpi=300)
    print("   ✅ Bar Chart saved.")

    # ==========================================
    # 5. VISUAL 2: NETWORK GRAPH (PURPLE THEME) 💜
    # ==========================================
    print("🕸️  2. Generating Network Graph...")

    ce_flat = [w for words in ce_keywords.values() for w in words]
    ai_flat = [w for words in ai_keywords.values() for w in words]
    all_specific_terms = ce_flat + ai_flat

    term_pairs = []
    for text in df['Combined_Text']:
        found_terms = [term for term in all_specific_terms if term in text]
        if len(found_terms) > 1:
            possible_pairs = list(itertools.combinations(set(found_terms), 2))
            for t1, t2 in possible_pairs:
                # Only connect CE terms to AI terms (Cross-domain)
                is_cross = (t1 in ce_flat and t2 in ai_flat) or (t1 in ai_flat and t2 in ce_flat)
                if is_cross:
                    term_pairs.append((t1, t2))

    pair_counts = Counter(term_pairs)
    filtered_pairs = {k: v for k, v in pair_counts.items() if v >= 2}
    top_pairs = sorted(filtered_pairs.items(), key=lambda x: x[1], reverse=True)[:35]

    if top_pairs:
        G = nx.Graph()
        for (term1, term2), weight in top_pairs:
            G.add_edge(term1, term2, weight=weight)

        plt.figure(figsize=(15, 12))
        pos = nx.spring_layout(G, k=2.5, seed=42)
        
        # Node Colors
        node_colors = ['#D6EAF8' if node in ce_flat else '#FADBD8' for node in G.nodes()]
        nx.draw_networkx_nodes(G, pos, node_color=node_colors, node_size=2800, edgecolors='#512E5F', linewidths=1.0)

        # Edges
        edges = G.edges(data=True)
        weights = [d['weight'] for u, v, d in edges]
        w_sorted = sorted(weights)
        p33 = np.percentile(w_sorted, 33)
        p66 = np.percentile(w_sorted, 66)

        tier_weak = []
        tier_medium = []
        tier_strong = []

        for u, v, d in edges:
            w = d['weight']
            if w >= p66: tier_strong.append((u, v))
            elif w >= p33: tier_medium.append((u, v))
            else: tier_weak.append((u, v))

        # Draw Edges (Purple Scale)
        nx.draw_networkx_edges(G, pos, edgelist=tier_weak, width=3.0, alpha=0.75, edge_color='#C39BD3')
        nx.draw_networkx_edges(G, pos, edgelist=tier_medium, width=5.0, alpha=0.85, edge_color='#8E44AD')
        nx.draw_networkx_edges(G, pos, edgelist=tier_strong, width=8.0, alpha=1.0, edge_color='#4A235A')

        nx.draw_networkx_labels(G, pos, font_size=11, font_weight='bold', bbox=dict(facecolor='white', alpha=0.85, edgecolor='none', pad=1))
        plt.title("Civil Engineering & AI: Connectivity Strength (Purple Scale)", fontsize=18)
        plt.axis('off')
        plt.tight_layout()
        plt.savefig(f"{OUTPUT_FOLDER}/2_Network_Graph_PURPLE_FINAL.png", dpi=300)
        print("   ✅ Network Graph saved.")

    # ==========================================
    # 6. VISUAL 3: WORD CLOUDS (WINNER TAKES ALL) ☁️
    # ==========================================
    print("☁️  3. Generating Word Clouds (Winner Takes All + Clean)...")

    # Function to find dominant category
    def get_dominant_category(text):
        scores = {area: 0 for area in ce_keywords.keys()}
        words_in_text = set(text.lower().split())
        for area, keywords in ce_keywords.items():
            match_count = len(words_in_text.intersection(keywords))
            scores[area] = match_count
        best_area = max(scores, key=scores.get)
        if scores[best_area] == 0: return None 
        return best_area

    df['Dominant_Area'] = df['Combined_Text'].apply(get_dominant_category)

    # Aggressive Stopwords List
    custom_stops = set(STOPWORDS)
    junk_words = [
        'fig', 'figs', 'figure', 'figures', 'data', 'dataset', 'database', 'model', 'modeling', 'models',
        'use', 'using', 'used', 'study', 'paper', 'method', 'result', 'analysis', 'analyze', 'performance',
        'system', 'application', 'approach', 'prediction', 'accuracy', 'time', 'cost', 'optimization',
        'development', 'based', 'proposed', 'new', 'value', 'image', 'including', 'shown', 'said', 'number',
        'design', 'construction', 'project', 'infrastructure', 'one', 'within', 'usd', 'set', 'make', 'solution',
        'monitoring', 'control', 'technology', 'tool', 'information', 'process', 'feature', 'parameter', 
        'show', 'provide', 'case', 'example', 'good', 'better', 'high', 'low', 'large', 'small', 'different',
        'real-time', 'efficiency', 'challenge', 'function', 'table', 'eq', 'civil', 'engineering'
    ]
    custom_stops.update(junk_words)

    for area in ce_keywords.keys():
        subset = df[df['Dominant_Area'] == area]
        if len(subset) > 5:
            text_pool = " ".join(subset['Combined_Text'].tolist())
            wc = WordCloud(width=900, height=500, background_color='white', stopwords=custom_stops, 
                           max_words=40, colormap='magma', collocations=False, random_state=42).generate(text_pool)
            
            plt.figure(figsize=(12, 6))
            plt.imshow(wc, interpolation='bilinear')
            plt.axis("off")
            plt.title(f"Word Cloud: {area} (Exclusive Articles)", fontsize=16, fontweight='bold', color='#4A235A')
            plt.tight_layout()
            plt.savefig(f"{OUTPUT_FOLDER}/3_WordCloud_{area}_FINAL.png", dpi=300)
            plt.close()
            print(f"   -> Word Cloud generated: {area}")
        else:
            print(f"      ⚠️ Not enough data for {area}.")

    print("\n✅ ALL VISUALIZATIONS COMPLETED!")